In [1]:
import torch

from dataloader_v1 import CIFData, gen_dataloaders
from qm9_model import Net, train, test

In [ ]:
train_dl, val_dl = gen_dataloaders('../data/dichalcogenides_public/cifs', 2500, 16)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.7, patience=5,
                                                       min_lr=0.00001)

In [ ]:
best_val_error = None
for epoch in range(1, 301):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(train_dl)
    val_error = test(val_dl)
    scheduler.step(val_error)

    if best_val_error is None or val_error <= best_val_error:
        test_error = test(val_dl)
        best_val_error = val_error

    print(f'Epoch: {epoch:03d}, LR: {lr:7f}, Loss: {loss:.7f}, '
          f'Val MAE: {val_error:.7f}, Test MAE: {test_error:.7f}')